In [1]:
from dotenv import load_dotenv
from loguru import logger
import sys

sys.path.append("..")

from news_extraction_pipeline.config import AINewsConfig
from news_extraction_pipeline.schemas import PipelineArgs
from news_extraction_pipeline.pipeline_steps import (
    extract_from_multiple_feed_urls,
    filter_by_date_threshold,
    filter_by_keywords,
    convert_datetime_columns_to_str
)

load_dotenv()

True

In [2]:
news_config = AINewsConfig()
pipeline_args = PipelineArgs()

In [3]:
bad_url = "https://techcrunch.com/category/artificial-intelligence/"

In [4]:
logger.info("Starting AI news retrieval process...")

news_config_dict = news_config.model_dump()

news_sources = [val for key, val in news_config_dict.items() if key.endswith("_FEED_URL")]

news_sources = [bad_url, news_config.MIT_NEWS_FEED_URL, news_config.AI_NEWS_FEED_URL]

articles = extract_from_multiple_feed_urls(news_sources)

2025-10-09 20:08:41.016 | INFO     | __main__:<module>:1 - Starting AI news retrieval process...
2025-10-09 20:08:41.020 | INFO     | news_extraction_pipeline.extractors.news.news_extractors:current_feed_url:64 - Setting current feed url to https://techcrunch.com/category/artificial-intelligence/
2025-10-09 20:08:41.022 | WARNING  | news_extraction_pipeline.extractor_selectors.extractor_selector:get_extractor:68 - No image extractor found for the url: https://techcrunch.com/category/artificial-intelligence/. The data retrieve will not contain an image_url
2025-10-09 20:08:41.022 | INFO     | news_extraction_pipeline.extractors.news.news_extractors:current_feed_url:64 - Setting current feed url to https://news.mit.edu/rss/feed
2025-10-09 20:08:41.022 | INFO     | news_extraction_pipeline.extractors.news.news_extractors:current_feed_url:64 - Setting current feed url to https://www.artificialintelligence-news.com/artificial-intelligence-news/feed/
2025-10-09 20:08:41.053 | INFO     | news

In [5]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62 entries, 0 to 49
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   title         62 non-null     object             
 1   news_link     62 non-null     object             
 2   image_link    62 non-null     object             
 3   publish_date  62 non-null     datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(3)
memory usage: 2.4+ KB


In [6]:
articles.sample(10)

,title,news_link,image_link,publish_date
17,AI maps how a new antibiotic targets gut bacteria,https://news.mit.edu/2025/ai-maps-how-new-anti...,https://news.mit.edu/sites/default/files/style...,2025-10-03 21:00:00+00:00
18,Secretary of Energy Chris Wright ’85 visits MIT,https://news.mit.edu/2025/secretary-energy-chr...,https://news.mit.edu/sites/default/files/style...,2025-10-03 04:00:00+00:00
27,System lets people personalize online social s...,https://news.mit.edu/2025/system-lets-people-p...,https://news.mit.edu/sites/default/files/style...,2025-10-01 14:00:00+00:00
13,Study sheds light on musicians’ enhanced atten...,https://news.mit.edu/2025/study-sheds-light-mu...,https://news.mit.edu/sites/default/files/style...,2025-10-06 19:45:00+00:00
29,3 Questions: Addressing the world’s most press...,https://news.mit.edu/2025/3-questions-mihaela-...,https://news.mit.edu/sites/default/files/style...,2025-09-30 12:00:00+00:00
0,Gemini Enterprise: Google aims to put an AI ag...,https://www.artificialintelligence-news.com/ne...,https://www.artificialintelligence-news.com/wp...,2025-10-09 12:00:03+00:00
36,How the brain splits up vision without you eve...,https://news.mit.edu/2025/how-brain-splits-vis...,https://news.mit.edu/sites/default/files/style...,2025-09-26 19:50:00+00:00
9,China Mobile Shanghai launches industry-first ...,https://www.artificialintelligence-news.com/ne...,https://www.artificialintelligence-news.com/wp...,2025-10-03 09:00:00+00:00
45,MIT engineers develop a magnetic transistor fo...,https://news.mit.edu/2025/mit-engineers-develo...,https://news.mit.edu/sites/default/files/style...,2025-09-23 15:32:00+00:00
1,Can Cisco’s new AI data centre router tackle t...,https://www.artificialintelligence-news.com/ne...,https://www.artificialintelligence-news.com/wp...,2025-10-09 09:00:00+00:00


In [7]:
filtered_by_date = filter_by_date_threshold(
    df = articles,
    filter_column=news_config.DATE_COLUMN,
    max_days_old=pipeline_args.max_days_old
)


2025-10-09 20:09:14.383 | INFO     | news_extraction_pipeline.pipeline_steps:filter_by_date_threshold:162 - Filtering articles published within the last 2 days.
2025-10-09 20:09:14.385 | INFO     | news_extraction_pipeline.pipeline_steps:filter_by_date_threshold:175 - Date filtering complete. 15 articles published within the allowed range.


In [8]:
filtered_by_kw = filter_by_keywords(
    df = filtered_by_date,
    filter_column=news_config.COLUMN_TO_FILTER_BY_KW,
    case_sen_search_kw=pipeline_args.case_sen_search_kw,
    case_insen_search_kw=pipeline_args.case_insen_search_kw,
)

2025-10-09 20:09:14.393 | DEBUG    | news_extraction_pipeline.pipeline_steps:filter_by_keywords:107 - Filtering articles by the next parameters...
2025-10-09 20:09:14.393 | DEBUG    | news_extraction_pipeline.pipeline_steps:filter_by_keywords:108 - case_sen_search_kw =[' AI ', 'AI ', 'AI ', 'A.I.', ' AI-', 'AI-']
2025-10-09 20:09:14.397 | DEBUG    | news_extraction_pipeline.pipeline_steps:filter_by_keywords:109 - case_insen_search_kw =['Artificial Intelligence', 'Machine Learning', 'Deep Learning', 'Neural Networks', 'NLP', 'Computer Vision', 'Data Science', 'Gemini', 'Bard', 'ChatGPT', 'GPT-4', 'DALL-E', 'MidJourney', 'Stable Diffusion', 'Claude', 'Whisper']
2025-10-09 20:09:14.397 | DEBUG    | news_extraction_pipeline.pipeline_steps:filter_by_keywords:110 - Filtering by column: title
2025-10-09 20:09:14.401 | INFO     | news_extraction_pipeline.pipeline_steps:filter_by_keywords:126 - Keyword filtering complete. 7 articles matched the criteria.


In [9]:
final_df = convert_datetime_columns_to_str(
    df = filtered_by_kw,
    string_format = news_config.DATE_STRING_FORMAT
)

final_df.info()

2025-10-09 20:09:14.407 | INFO     | news_extraction_pipeline.pipeline_steps:convert_datetime_columns_to_str:214 - Converting datetime columns to string format: %Y-%m-%dT%H:%M:%SZ
2025-10-09 20:09:14.414 | DEBUG    | news_extraction_pipeline.pipeline_steps:convert_datetime_columns_to_str:215 - Detected datetime columns: ['publish_date']
2025-10-09 20:09:14.416 | INFO     | news_extraction_pipeline.pipeline_steps:convert_datetime_columns_to_str:220 - Datetime conversion complete.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         7 non-null      object
 1   news_link     7 non-null      object
 2   image_link    7 non-null      object
 3   publish_date  7 non-null      object
dtypes: object(4)
memory usage: 356.0+ bytes


## Full Pipeline in a line

In [10]:
import sys

sys.path.append("..")

from news_extraction_pipeline.pipeline import main

final_articles = main()

2025-10-09 20:09:14.431 | INFO     | news_extraction_pipeline.pipeline:main:34 - Starting AI news retrieval process...
2025-10-09 20:09:14.433 | INFO     | news_extraction_pipeline.extractors.news.news_extractors:current_feed_url:64 - Setting current feed url to https://news.mit.edu/rss/feed
2025-10-09 20:09:14.438 | INFO     | news_extraction_pipeline.extractors.news.news_extractors:current_feed_url:64 - Setting current feed url to https://www.artificialintelligence-news.com/artificial-intelligence-news/feed/
2025-10-09 20:09:14.438 | INFO     | news_extraction_pipeline.extractor_selectors.extractor_selector:get_extractor:73 - Extractor 'MITImageExtractor' selected for base URL: https://news.mit.edu
2025-10-09 20:09:14.440 | INFO     | news_extraction_pipeline.extractor_selectors.extractor_selector:get_extractor:73 - Extractor 'AINEWSImageExtractor' selected for base URL: https://www.artificialintelligence-news.com
2025-10-09 20:09:32.792 | INFO     | news_extraction_pipeline.extracto

In [11]:
final_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         7 non-null      object
 1   news_link     7 non-null      object
 2   image_link    7 non-null      object
 3   publish_date  7 non-null      object
dtypes: object(4)
memory usage: 356.0+ bytes
